In [0]:
cd /content/drive/My Drive/kaggle/Bengali.AI competition

/content/drive/My Drive/kaggle/Bengali.AI competition


In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

이전과 다르게 이미지를 3차원으로 바꿔준다.

In [0]:
import joblib

img = joblib.load('data/Train_50209.pkl')
img = img.reshape(137, 236).astype(np.int8)
img = 256 - img
img = img[:,:, np.newaxis]
img = np.repeat(img, 3, 2) # 3차원으로 만들기 위해서는 이렇게 해야한다.

In [0]:
!pip install albumentations

     |████████████████████████████████| 634kB 2.7MB/s 
  Created wheel for imgaug: filename=imgaug-0.2.6-cp36-none-any.whl size=654020 sha256=491b5faa97953647a8ca6dd973a81acd8111ee80168b2f32e0b1f88f7dd87b88
  Stored in directory: /root/.cache/pip/wheels/97/ec/48/0d25896c417b715af6236dbcef8f0bed136a1a5e52972fc6d0
Successfully built imgaug
  Found existing installation: imgaug 0.2.9
    Uninstalling imgaug-0.2.9:
      Successfully uninstalled imgaug-0.2.9


In [0]:
from albumentations import (
    HorizontalFlip, IAAPerspective, ShiftScaleRotate, CLAHE, RandomRotate90,
    Transpose, ShiftScaleRotate, Blur, OpticalDistortion, GridDistortion, HueSaturationValue,
    IAAAdditiveGaussianNoise, GaussNoise, MotionBlur, MedianBlur, RandomBrightnessContrast, IAAPiecewiseAffine,
    IAASharpen, IAAEmboss, Flip, OneOf, Compose
)

from albumentations.pytorch import ToTensor

In [0]:
# 넣어주는 확률에 따라 이미지 변환
train_transform = Compose([
        RandomRotate90(),
        Flip(),
        Transpose(),
        OneOf([ 
            IAAAdditiveGaussianNoise(),
            GaussNoise(),
        ], p=0.2),
        OneOf([
            MotionBlur(p=0.2),
            MedianBlur(blur_limit=3, p=0.1),
            Blur(blur_limit=3, p=0.1),
        ], p=0.2),
        ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.2, rotate_limit=45, p=0.2),
        OneOf([
            OpticalDistortion(p=0.3),
            GridDistortion(p=.1),
            IAAPiecewiseAffine(p=0.3),
        ], p=0.2),
        OneOf([
            CLAHE(clip_limit=2),
            IAASharpen(),
            IAAEmboss(),
            RandomBrightnessContrast(),            
        ], p=0.3),
        HueSaturationValue(p=0.3),
        ToTensor() # pytorch에서 channel, width, height 이므로 저절로 이렇게 맞춰준다
    ], p=1.0)

In [0]:
from torch.utils.data import Dataset 

# transform 까지 추가해준다
class BengaliDataset(Dataset):
    def __init__(self, csv, img_height, img_width, transform):
        self.csv = csv.reset_index()
        self.img_ids = csv['image_id'].to_numpy()
        self.img_height = img_height
        self.img_width = img_width
        self.transform = transform

    def __len__(self):
        return len(csv)

    def __getitem__(self, index):
        img_id = self.img_ids[index]
        img = joblib.load(f'data/train_image/{img_id}.pkl')
        img = img.reshape(self.img_height, self.img_width).astype(np.uint8)
        img = 255 - img
        img = img[:, :, np.newaxis]
        img = np.repeat(img, 3, 2)
        if self.transform is not None:
            img = self.transform(image=img)['image']
        
        label_1 = self.csv.iloc[index].grapheme_root
        label_2 = self.csv.iloc[index].vowel_diacritic
        label_3 = self.csv.iloc[index].consonant_diacritic

        return img, label_1, label_2, label_3